In [ ]:
""" 
    Needless to say, the notebook is incomplete. It does not have `classification_train_loader`, 
    `classification_test_loader`, `embedding_model`. However, the main idea of this file is just to ensure
    that we get hold of the KNN in a fast manner
"""

In [ ]:
# Let us save the embedding of all elements in the train_data as well as their index
# this can be used later for our purpose
embedding_space = []
for i, data in enumerate(classification_train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        batch_size = inputs.shape[0]
        inputs = inputs.reshape(batch_size,-1)
        labels = labels.cuda()
        embeddings = embedding_model(inputs)
        for index,tensor in enumerate(embeddings):
            embedding_space.append((labels[index],tensor))

In [ ]:
"""
    We should get hold of the tensor which is placed in the second
    dimension of the tuple. Just put it in a list
"""
dist = [space for index,space in embedding_space[:-1]]
"""
    List of tensor handling is not easy and I often struggled with how exactly to 
    convert it into a matrix. Turns out, doing a simple `stack` was sufficient.
    Now we have a matrix that we can work upon
"""
result = torch.stack(dist, dim=0)
result.shape

In [ ]:
"""
    Note how we move from the list of tuples to the distance embeddings.
    Luckily, we have the same set of values at both the ends which is 
    really helpful and we can directly index them
"""
from collections import Counter
def k_nearest_neighbour(distance_embeddings, vector, k=3):   
    updated_result = torch.norm((distance_embeddings - vector),p=2,dim=1)
    values, indices = torch.topk(updated_result,k=k ,largest=False)
    candidate_indexes = []
    for index in indices:
        candidate_indexes.append((embedding_space[index][0]).item())
    return Counter(candidate_indexes).most_common(1)[0][0]

In [ ]:
correct = 0
total = 0
for data in classification_test_loader:
    inputs, labels = data
    inputs = inputs.cuda()
    batch_size = inputs.shape[0]
    inputs = inputs.reshape(batch_size,-1)
    # Next the evaluation
    output = embedding_model.fc(inputs)
    for idx,query in enumerate(output):
        predicted = k_nearest_neighbour(result, query, 1)
        correct += (predicted == labels[idx])
    total += labels.size(0)
    correct = correct.cpu().sum()
print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct /total))